In [66]:
# Modelling

%run ../notebook_preamble.ipy

import logging
import pandas as pd
import numpy as np
import altair as alt
import sg_covid_impact
from sg_covid_impact.getters.nomis import get_BRES
from sg_covid_impact.complexity import *
from sg_covid_impact.secondary_data import read_secondary
from sg_covid_impact.descriptive import (
    read_official,read_lad_nuts1_lookup,assign_nuts1_to_lad,
    read_claimant_counts,claimant_count_norm,read_search_trends,make_exposure_shares,
                                         search_trend_norm,make_high_exposure,rank_sector_exposures,
                                        make_exposure_shares_detailed)
from sg_covid_impact.make_sic_division import extract_sic_code_description,load_sic_taxonomy
from sg_covid_impact.diversification import (
    load_predicted,extract_sectors,extract_network,make_diversification_options,make_sector_space_base)
from sg_covid_impact.utils.altair_save_utils import *
import statsmodels.api as sm

project_dir = sg_covid_impact.project_dir


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
driver = google_chrome_driver_setup()

[WDM] - Current google-chrome version is 87.0.4280


 
2021-01-15 07:07:44,418 - WDM - INFO - Current google-chrome version is 87.0.4280


[WDM] - Get LATEST driver version for 87.0.4280


2021-01-15 07:07:44,424 - WDM - INFO - Get LATEST driver version for 87.0.4280


[WDM] - Get LATEST driver version for 87.0.4280


2021-01-15 07:07:44,568 - WDM - INFO - Get LATEST driver version for 87.0.4280


[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_mac64.zip


2021-01-15 07:07:44,619 - WDM - INFO - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/jmateosgarcia/.wdm/drivers/chromedriver/mac64/87.0.4280.88]


2021-01-15 07:07:45,780 - WDM - INFO - Driver has been saved in cache [/Users/jmateosgarcia/.wdm/drivers/chromedriver/mac64/87.0.4280.88]


In [5]:
def make_exposure_share_variable(exposure_thres=7):

    logging.info("Calculating sector exposure")
    exposures_ranked = make_sector_exposure_table()  

    logging.info("Calculating local exposure shares")
    exposure_lad_codes = make_local_exposure_table(exposures_ranked)
    
    logging.info(f"Calculating high exposure shares level {exposure_thres}")
    exposure_high = (make_high_exposure(exposure_lad_codes,geo='geo_cd',
                                       level=exposure_thres)
                     .assign(variable='exposure_share')
                     .rename(columns={'share':'value'}))
    
    return exposure_high

def make_sector_exposure_table():
    
    d = read_search_trends()
    trends_normalised = search_trend_norm(d)
    exposures_ranked = rank_sector_exposures(trends_normalised,'division')
    
    return exposures_ranked

def make_local_exposure_table(exposures_ranked):
    
    bres = read_official()
    exposure_levels = exposures_ranked.merge(bres,left_on='division',right_on='division')
    exposure_lad_codes = make_exposure_shares(exposure_levels,'geo_cd')
    return exposure_lad_codes


def make_div_share_variable(exposure_level = 7, div_level = 2):
    
    _DIVISION_NAME_LOOKUP = extract_sic_code_description(load_sic_taxonomy(), "Division")
    
    logging.info("Calculating sector exposure")
    exposures_ranked = make_sector_exposure_table()
    division_month_exposure_dict = exposures_ranked.set_index(['division','month'])['rank'].to_dict()

    logging.info("Making sector space")
    my_divisions = list(set(exposures_ranked['division']))
    pr = load_predicted()
    pr_selected = pr[my_divisions]
    t = extract_sectors(pr_selected,0.5)
    
    div_space = extract_network(t)
    p,g,l = make_sector_space_base(sector_space=div_space,extra_edges=70)
    
    
    logging.info("Calculating local exposure shares")
    bres = read_official()
    exposure_levels = exposures_ranked.merge(bres,left_on='division',right_on='division')
    exposure_levels['division_name'] = exposure_levels['division'].map(_DIVISION_NAME_LOOKUP)
    exposure_shares_detailed = make_exposure_shares_detailed(exposure_levels,geo='geo_cd')
    
    logging.info("Calculating diversification share rankings")
    monthly_diversification_rankings = pd.concat([
        (make_diversification_options(
            g,division_month_exposure_dict,m,range(exposure_level,10),[0,1,2,3])
     .sort_values('mean',ascending=False)
     .assign(divers_ranking = lambda x: pd.qcut(x['mean'],q=np.arange(0,1.1,0.25),labels=False,
                                               duplicates='drop'))
     .assign(month=m)) for m in range(3,11)])
    
    # Merge with diversification information
    logging.info(f"Calculating diversification shares level {str(div_level)}")
    diversification_lad_detailed = (exposure_levels
                                    .merge(monthly_diversification_rankings,
                                      left_on=['division','month'],
                                      right_on=['division','month'],
                                      how='outer'))

    diversification_lad_detailed['divers_ranking'] = (diversification_lad_detailed
                                                      ['divers_ranking']
                                                      .fillna('Less exposed'))
    
    diversification_shares = (make_exposure_shares(
        diversification_lad_detailed,geography='geo_cd',variable='divers_ranking')
                              .query(f"divers_ranking == {div_level}")
                              .assign(variable='low_diversification_share')
                              .drop(axis=1,labels=['value'])
                              .rename(columns={'share':'value'})[
                                  ['month','geo_cd','variable','value']]).reset_index(drop=True)
    
    return diversification_shares
    
def make_claimant_count_variable():
    '''
    '''
    cl = read_claimant_counts()
    cl_count = (cl
            .query("measure_name=='Claimants as a proportion of residents aged 16-64'")
            [['geography_code','month','obs_value']]
            .assign(variable='cl_count')
            .rename(columns={'obs_value':'value',
                            'geography_code':'geo_cd'}))
    
    cl_norm_ = claimant_count_norm(cl)
    
    cl_norm = (cl_norm_[['geography_code','month','cl_norm']]
           .rename(columns={'geography_code':'geo_cd',
                           'cl_norm':'value'})
           .assign(variable='cl_count_norm'))
    
    return pd.concat([cl_count,cl_norm])
    
def make_secondary_variables():
    
    secondary = read_secondary()
    
    secondary_out = (secondary
                     .rename(columns={'geography_code':'geo_cd'})
                     [['geo_cd','variable','value']])
    
    return secondary_out
    

In [6]:
exp = make_exposure_share_variable()

2021-01-13 16:04:55,044 - root - INFO - Calculating sector exposure
2021-01-13 16:04:58,073 - root - INFO - Calculating local exposure shares
2021-01-13 16:05:03,618 - root - INFO - Calculating high exposure shares level 7


In [7]:
div = make_div_share_variable()

2021-01-13 16:05:03,724 - root - INFO - Calculating sector exposure
2021-01-13 16:05:06,665 - root - INFO - Making sector space
2021-01-13 16:05:44,169 - root - INFO - Calculating local exposure shares
2021-01-13 16:06:11,557 - root - INFO - Calculating diversification share rankings
2021-01-13 16:06:11,666 - root - INFO - Calculating diversification shares level 2


In [8]:
cl = make_claimant_count_variable()

In [9]:
secondary = make_secondary_variables()

In [10]:
bres_sic_wide = get_BRES().pivot_table(index='geo_cd',columns='SIC4',values='value')

compl = (calc_eci(create_lq(bres_sic_wide,binary=True)).
         reset_index(drop=False)
         .assign(variable='eci')
         .rename(columns={'eci':'value'})
         .assign(value=lambda x: -x.value))
secondary = pd.concat([secondary,compl])

2021-01-13 16:06:18,198 - sg_covid_impact.complexity - WARNING - Dropping all zero cols: ['0111' '0112' '0113' '0114' '0115' '0116' '0119' '0121' '0122' '0123'
 '0124' '0125' '0126' '0127' '0128' '0129' '0130' '0141' '0142' '0143'
 '0144' '0145' '0146' '0147' '0149' '0150' '0520' '0710' '0721' '0729'
 '1104' '5122' '6530' '9700' '9810' '9820' '9900']
2021-01-13 16:06:18,328 - sg_covid_impact.complexity - INFO - CI sign: 1.0


In [11]:
_SHORT_VAR_NAMES = {'% with NVQ4+ - aged 16-64':'% tertiary',
                          '% with no qualifications (NVQ) - aged 16-64':'% no qual',
                          '% with other qualifications (NVQ) - aged 16-64':'% other qual',
                          'Annual pay - gross':'Gross annual pay',
                          'Economic activity rate - aged 16-64':'Econ Activity rate',
                          'Employment rate - aged 16-64': 'Emp rate',
                          'cl_count':'Claimant count',
                          'cl_count_norm':'Claimant (normalised)',
                          'smd_high_deprivation_share':'SMDI',
                          'eci':'ECI'}

sort_vars = ['Claimant count','Claimant (normalised)','ECI','Emp rate','Gross annual pay',
             '% tertiary','% no qual']

In [12]:
### Visualise relationships between variables

def make_correlation_df(web_vars,other_vars):
    '''Calculates correlations between exposure / diversification variables and secondary data
    '''
    out = []

    for w in web_vars:
        for o in other_vars:
            if 'month' in o.columns:
                for v in set(o['variable']):
                    for m in set(w['month']):
                        my_w = w.query(f"month=={m}")
                        my_o = (o
                                .query(f"month=={m}")
                                .query(f"variable == '{v}'"))
                        merg = my_w.merge(my_o,on='geo_cd')
                        my_v_name = list(set(w['variable']))[0]
                        corr = np.float(merg[['value_x','value_y']].corr().iloc[0,1])
                        res = pd.Series([my_v_name,v,m,corr],
                                        index=['primary','secondary','month','corr'])
                        out.append(res)
            else:
                for v in set(o['variable']):
                    for m in set(w['month']):
                        my_w = w.query(f"month=={m}")
                        my_o = (o
                                .query(f"variable == '{v}'"))
                        merg = my_w.merge(my_o,on='geo_cd')
                        my_v_name = list(set(w['variable']))[0]
                        corr = np.float(merg[['value_x','value_y']].corr().iloc[0,1])
                        res = pd.Series([my_v_name,v,m,corr],
                                        index=['primary','secondary','month','corr'])
                        out.append(res)
                    
    correlation_df = pd.DataFrame(out)  

    correlation_df = (correlation_df
                      .assign(secondary_short = lambda x: x['secondary']
                              .map(_SHORT_VAR_NAMES)))

    return correlation_df

In [13]:
web_vars = [div,exp]
other_vars = [cl,secondary]

correlation_df= make_correlation_df(web_vars,other_vars)
correlation_df = (correlation_df
                  .loc[~correlation_df['secondary_short']
                       .isin(['% other qual','SMDI','Econ Activity rate'])])

In [14]:
ch = (alt.Chart(correlation_df.query("month>=3"))
      .mark_point(filled=True)
      .encode(
      x=alt.X('month',scale=alt.Scale(domain=[2.9,10])),
      y='corr',
      facet=alt.Facet('secondary_short',columns=2,
                  sort=sort_vars,
                  header=alt.Header(labelAngle=0,labelAnchor='start',
                                                     labelOrient='top')),
      color='primary')).properties(width=300,height=50)

In [15]:
ch

alt.Chart(...)

In [16]:
def make_geo_average(ser):
    return (ser
            .pivot_table(index='geo_cd',
                        columns='variable',
                        values='value',aggfunc='mean'))

In [17]:
all_vars_df_wide = pd.concat([make_geo_average(x) for x in [div,exp,cl,secondary]],axis=1)

all_vars_df_corr = all_vars_df_wide.corr()

exp_div_corr = (all_vars_df_corr[['low_diversification_share','exposure_share']]
                .drop(
                    index=['cl_count','cl_count_norm','low_diversification_share','exposure_share',
                           'smd_high_deprivation_share','% with other qualifications (NVQ) - aged 16-64'])
                .reset_index(drop=False)
                .melt(id_vars='variable',var_name='exposure_measure')
               )
exp_div_corr['v'] =0

base = alt.Chart(exp_div_corr).properties(width=150,height=200)


out = (base
       .mark_point(filled=True,size=75,stroke='black',strokeWidth=0.5)
       .encode(y=alt.Y('variable',title='Variable',
                       sort=alt.EncodingSortField('value',op='min',order='descending')),
              x='value',
              color=alt.Color('exposure_measure',
                              #scale=alt.Scale(scheme='spectral'))
                             )))
out_lines = (base
       .mark_line(stroke='grey',strokeWidth=1,strokeDash=[2,1])
       .encode(y=alt.Y('variable',title='Variable',
                       sort=alt.EncodingSortField('value',op='min',order='descending')),
              x='value',
              #color='exposure_measure',
              detail='variable')
      )

vline = (base
        .mark_rule()
        .encode(x=alt.X('v',title='Correlation coefficient')))

ch = (out+out_lines+vline)

ch

alt.LayerChart(...)

In [18]:
_LAD_NUTS_LU = read_lad_nuts1_lookup()

exp_wide =  (
    pd.concat([make_geo_average(x) for x in [div,exp]],axis=1)
    .reset_index(drop=False)
    .assign(nuts1 = lambda x: x['geo_cd'].apply(assign_nuts1_to_lad))
    .melt(id_vars=['geo_cd','nuts1']))

sorted_nuts = (exp_wide
               .query("variable=='low_diversification_share'")
               .groupby('nuts1')['value']
               .mean()
               .sort_values(ascending=False)
               .index.tolist())

bp = (alt.Chart(exp_wide)
      .mark_boxplot()
      .encode(x='value',
             y=alt.Y('nuts1',sort=sorted_nuts),
             column='variable',
             color=alt.Color('variable',legend=None))).properties(height=200,width=150)
bp

alt.Chart(...)

In [19]:
geo_diffs = alt.vconcat(bp,ch).resolve_scale(color='independent')
save_altair(geo_diffs,'geo_comparisons',driver)

geo_diffs

/Users/jmateosgarcia/Desktop/projects/sg_covid_impact/figures
2021-01-13 16:06:21,740 - tornado.access - INFO - 200 GET / (::1) 1.43ms
2021-01-13 16:06:21,782 - tornado.access - INFO - 200 GET /vega.js (::1) 3.83ms
2021-01-13 16:06:21,784 - tornado.access - INFO - 200 GET /vega-embed.js (::1) 5.40ms
2021-01-13 16:06:21,786 - tornado.access - INFO - 200 GET /vega-lite.js (::1) 3.18ms
2021-01-13 16:06:21,932 - tornado.access - WARNING - 404 GET /favicon.ico (::1) 1.87ms


alt.VConcatChart(...)

### Run regression

In [20]:
def make_lagged_web(var,name):
    
    results = []
    
    var_ = var.query("month>=3")
    
    for m in set(var_['month']):
        pre = var_.query(f"month<{m}")
        stat = pre.groupby(['geo_cd','variable'])['value'].mean()
        stat.name=m
        #stat.assign(month=m)
        results.append(stat)
        
    lagged =  (pd.concat(results,axis=1).loc[:,range(4,11)]
               .reset_index(drop=False)
               .melt(id_vars=['geo_cd','variable'],var_name='month')
               .drop(axis=1,labels=['variable'])
               .rename(columns={'value':f'{name}_lagged'})
               .set_index(['geo_cd','month']))
    return lagged

def make_secondary_reg(keep):
    
    sec_wide = make_geo_average(secondary)
    sec_wide.columns = [_SHORT_VAR_NAMES[x] for x in sec_wide.columns]

    
    return sec_wide[keep]

In [21]:
_SEC_KEEP = ['% tertiary','% no qual','Gross annual pay','Emp rate','ECI']

In [53]:
def make_regression_table(secondary,keep=['% tertiary','Gross annual pay','Emp rate','ECI']):
    
    X = (cl.copy()
         .query("month>3")
         .pivot_table(index=['geo_cd','month'],columns='variable',values='value'))
    
    # Present period
    present = pd.concat(
        [var.rename(columns={'value':f'{name}_present'})
         .query("month>3")
         .set_index(['month','geo_cd'])
         .drop(axis=1,labels=['variable']) for
        var,name in zip([exp,div],['exposure_share','low_div_share'])],axis=1).reset_index(drop=False)
    
    lagged = pd.concat([make_lagged_web(v,name) for v,name in zip([exp,div],['exposure_share','low_div_share'])],
                      axis=1).reset_index(drop=False)
    
    sec_vars = make_secondary_reg(keep)
    
    data = (X
            .merge(present,on=['geo_cd','month'])
            .merge(lagged,on=['geo_cd','month'])
            .merge(sec_vars,on='geo_cd'))
    #data = pd.concat([data,pd.get_dummies(data['geo_cd'])],axis=1)
    
    return data
    

In [54]:
reg_table = make_regression_table(secondary)

In [62]:
def fit_regression(table,dep,indep_focus,fe=True):
    
    table_ = table.dropna(axis=0)
    
    Y = table_[dep]
    
    indep = table_[[x for x in table_.columns if indep_focus in x]]
    
    other_vars = table_.drop(columns=['cl_count','cl_count_norm',
                                     'exposure_share_present','exposure_share_lagged',
                                     'low_div_share_present','low_div_share_lagged','geo_cd'])    
    
    fe = pd.get_dummies(table_['geo_cd'])
    
    
    exog = pd.concat([indep,
                      other_vars,
                      fe
                     ],axis=1)
    
    #exog=sm.add_constant(exog)
    lm = sm.OLS(endog=Y,exog=exog)
    return lm.fit(cov_type='HC2')
    #return exog

def extract_model_results(model,name):
    '''Model
    '''
    res = pd.concat([model.params,model.conf_int()],axis=1)
    res.columns = ['coefficient','min','max']
    res['indep']='_'.join(name.split('_')[:-1])
    
    return res

In [63]:
#out = fit_regression(reg_table,'cl_count','exp')

mods = {}

for dep in ['cl_count','cl_count_norm']:
    for indep in ['exp','div']:
        mods[f'{dep}_{indep}'] = fit_regression(reg_table,dep,indep)
        
model_results = pd.concat(
    [extract_model_results(v,name=k) for k,v in mods.items()])    

my_vars = ['exposure_share_present','exposure_share_lagged',
          'low_div_share_present','low_div_share_lagged']

In [64]:
model_selected = (model_results
                   .reset_index(drop=False)
                   .loc[model_results.index.isin(my_vars)]).reset_index(drop=True)

model_selected['pred'] = ['exp share' if 'exposure' in x else 'low div share' for x in model_selected['index']]
model_selected['temporal'] = ['lagged' if 'lagged' in x else 'present' for x in model_selected['index']]

In [68]:
base = (alt.Chart()
     .encode(y=alt.Y('temporal',sort=['present','lagged'])))


ch = (base
     .mark_bar()
     .encode(
         x='coefficient',
         color=alt.Color('temporal',legend=None)
            )
     ).properties(width=120,height=75)
err = (base
     .mark_errorbar(color='black')
     .encode(
         x=alt.X('min',title='Coefficient'),
         x2='max')
     ).properties(width=120,height=75)

reg_plot = alt.layer(ch,err,data=model_selected).facet(column=alt.Column(
    'indep',title='Independent variable'),
                                            row=alt.Row('pred',sort=['exp','diversification'],
                                                       title='Measure of exposure'))

save_altair(reg_plot,'coeff_plot',driver)

reg_plot

/Users/jmateosgarcia/Desktop/projects/sg_covid_impact/figures
2021-01-15 07:07:53,177 - tornado.access - INFO - 200 GET / (::1) 2.84ms
2021-01-15 07:07:53,219 - tornado.access - INFO - 200 GET /vega-embed.js (::1) 1.15ms
2021-01-15 07:07:53,223 - tornado.access - INFO - 200 GET /vega.js (::1) 4.96ms
2021-01-15 07:07:53,229 - tornado.access - INFO - 200 GET /vega-lite.js (::1) 7.95ms
2021-01-15 07:07:53,354 - tornado.access - WARNING - 404 GET /favicon.ico (::1) 0.59ms


alt.FacetChart(...)

In [69]:
mods['cl_count_norm_div'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          cl_count_norm   R-squared:                       0.878
Model:                            OLS   Adj. R-squared:                  0.857
Method:                 Least Squares   F-statistic:                       nan
Date:                Fri, 15 Jan 2021   Prob (F-statistic):                nan
Time:                        07:07:57   Log-Likelihood:                 191.70
No. Observations:                2485   AIC:                             332.6
Df Residuals:                    2127   BIC:                             2415.
Df Model:                         357                                         
Covariance Type:                  HC2                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
low_div_share_present     1.8543      0.195      9.490      0.000       1.471       2.237
low_div_share_lagged      9.7238      0.827     11.760      0.000       8.103      11.344
month                     0.0004      0.003      0.135      0.892      -0.005       0.006
% tertiary                0.0070      0.001      9.149      0.000       0.005       0.008
Gross annual pay       2.474e-05   1.65e-06     15.010      0.000    2.15e-05     2.8e-05
Emp rate                  0.0125      0.001     17.184      0.000       0.011       0.014
ECI                       0.1402      0.009     15.860      0.000       0.123       0.158
E06000001                -0.5840      0.037    -15.898      0.000      -0.656      -0.512
E06000002                -0.1290      0.015     -8.665      0.000      -0.158      -0.100
E06000003                -0.1828      0.043     -4.289      0.000      -0.266      -0.099
E06000004                -0.5059      0.024    -21.100      0.000      -0.553      -0.459
E06000005                -0.6188      0.036    -17.224      0.000      -0.689      -0.548
E06000006                -0.4614      0.024    -18.998      0.000      -0.509      -0.414
E06000007                -0.3790      0.055     -6.877      0.000      -0.487      -0.271
E06000008                -0.3564      0.057     -6.255      0.000      -0.468      -0.245
E06000009                -0.2992      0.065     -4.611      0.000      -0.426      -0.172
E06000010                -0.1897      0.029     -6.486      0.000      -0.247      -0.132
E06000011                -0.0731      0.043     -1.714      0.086      -0.157       0.010
E06000012                -0.3725      0.023    -16.013      0.000      -0.418      -0.327
E06000013                -0.3991      0.039    -10.237      0.000      -0.475      -0.323
E06000014                 0.2787      0.064      4.349      0.000       0.153       0.404
E06000015                -0.0706      0.078     -0.901      0.367      -0.224       0.083
E06000016                 0.2767      0.064      4.332      0.000       0.152       0.402
E06000017                 0.4885      0.140      3.479      0.001       0.213       0.764
E06000018                -0.2498      0.028     -8.897      0.000      -0.305      -0.195
E06000019                -0.0233      0.064     -0.362      0.717      -0.149       0.103
E06000020                 0.2138      0.050      4.253      0.000       0.115       0.312
E06000021                -0.2752      0.033     -8.422      0.000      -0.339      -0.211
E06000022                -0.3043      0.062     -4.890      0.000      -0.426      -0.182
E06000023                -0.1182      0.074     -1.601      0.109      -0.263       0.026
E06000024                -0.2272      0.055     -4.145      0.000      -0.335      -0.120
E06000025              